In [1]:
using Pkg
Pkg.activate(".")

 Activating environment at `/mnt/c/Users/mathi/Git/jump-dev_2020/Project.toml`


In [2]:
using LinearAlgebra
using SparseArrays

using Quadmath

using Tulip
using Tulip.TLA
using Tulip.KKT

const Krylov = Tulip.KKT.Krylov

Krylov

# Design and implementation of [Tulip.jl](https://github.com/ds4dm/Tulip.jl)

<br />

Mathieu Tanneau

JuMP developper call - Aug. 28th, 2020

<br />
  

Technical report: https://arxiv.org/pdf/2006.08814.pdf <br />
Material: https://github.com/mtanneau/jump-dev_2020

## Shameless advertising

I'll be defending on October 2nd.
* Decomposition techniques in power systems
* Structured interior-point methods
* Disjunctive cuts in mixed-integer conic optimization

Details coming at https://cerc-datascience.polymtl.ca/

## Overview

This talk is about
1. The motivation between Tulip
2. Design choices to allow specialized linear algebra
3. Some lessons learned about internal data structures

# Motivation

Linear programming problems:
* linear objective
* linear constraints
* all variables continuous

In *standard form*:

\begin{align}
    (LP) \ \ \ \min_{x} \ \ \ 
    & c^{T}x\\
    & A x = b\\
    & x \geq 0
\end{align}

Typically solved with simplex or interior-point method (IPM).

~~Almost~~ all IPM algorithms end solve _augmented systems_

$$
    \begin{bmatrix}
        -\Theta^{-1} & A^{T}\\
        A & 0
    \end{bmatrix}
    \begin{bmatrix}
        \Delta x\\
        \Delta y
    \end{bmatrix}
    =
    \begin{bmatrix}
        b\\
        c
    \end{bmatrix}
$$

* Symmetric, indefinite
* Can be reduced to normal equations $(A \Theta A^{T}) \Delta y = \xi$

Tulip: _regularized_ augmented system

$$
    \begin{bmatrix}
        -(\Theta^{-1} + {\color{red}{R_{p}}}) & A^{T}\\
        A & {\color{red}{R_{d}}}
    \end{bmatrix}
    \begin{bmatrix}
        \Delta x\\
        \Delta y
    \end{bmatrix}
    =
    \begin{bmatrix}
        b\\
        c
    \end{bmatrix}
$$
where $R_{p}, R_{d} \succ 0$. (see paper for the mathematical justification)

* Symmetric quasi-definite (SQD)
* Amenable to $LDL^{T}$-type factorizations

Design of Tulip: exploit structure of $A$ to speed-up the resolution of

$$
    \begin{bmatrix}
        -(\Theta^{-1} + R_{p}) & A^{T}\\
        A & R_{d}
    \end{bmatrix}
    \begin{bmatrix}
        \Delta x\\
        \Delta y
    \end{bmatrix}
    =
    \begin{bmatrix}
        b\\
        c
    \end{bmatrix}
$$

E.g.:
* Efficient data structures
* Specialized matrix-vector product
* Specialized factorization routines

3 requirements to do that in practice:
1. Develop structure-exploiting data structures & routines...
2. ... and integrate them in the IPM algorithmic framework
3. Convey structure information to the solver

This presentation: 2. & 3.

# Some examples

## Using a different arithmetic

The model's arithmetic is chosen at instantiation.

In [3]:
tlp = Tulip.Model{Float64}()  # Double-precision
Tulip.load_problem!(tlp, "dat/afiro.mps")
Tulip.optimize!(tlp)
Tulip.get_attribute(tlp, Tulip.ObjectiveValue())

-464.75314208040555

In [4]:
tlp = Tulip.Model{Float128}()  # Quadruple-precision
Tulip.load_problem!(tlp, "dat/afiro.mps")
Tulip.optimize!(tlp)
Tulip.get_attribute(tlp, Tulip.ObjectiveValue())

-4.64753142857142852723921775450696180e+02

Also available in MOI with `Tulip.Optimizer{T}`
```julia
moi64  = Tulip.Optimizer{Float64}()   # double precision
moi128 = Tulip.Optimizer{Float128}()  # quad precision
```

## Using specialized linear algebra

In [5]:
tlp = Tulip.Model{Float64}()  # Double-precision
Tulip.load_problem!(tlp, "dat/afiro.mps")
tlp.params.OutputLevel = 1;

The user can specify:
* Which data structure for $A$ (via `MatrixOptions`)
* Which linear solver (via `KKTOptions`), i.e.,
  * Augmented system vs normal equations
  * Direct vs Indirect
  * Backend

Multiple dispatch takes care of the rest.

In [6]:
# Dense linear algebra
tlp.params.MatrixOptions = Tulip.TLA.MatrixOptions(Matrix)
tlp.params.KKTOptions    = KKT.SolverOptions(KKT.Dense_SymPosDef);

Information about the linear solver is displayed in the log
```
Linear solver options
  Arithmetic   : Float64
  Backend      : LAPACK
  System       : Normal equations
```

In [7]:
Tulip.optimize!(tlp)


Problem info
  Name        : AFIRO
  Constraints : 27
  Variables   : 32
  Non-zeros   : 83

Reduced problem info
  Constraints : 25  (removed 2)
  Variables   : 32  (removed 0)
  Non-zeros   : 81  (removed 2)
Presolve time : 0.000s

Optimizer info
Linear solver options
  Arithmetic   : Float64
  Backend      : LAPACK
  System       : Normal equations

 Itn            PObj            DObj     PFeas    DFeas    GFeas       Mu  Time
   0  +8.5491918e+00  -6.2055378e+02  5.21e+02 9.00e+00 6.30e+02  1.0e+00  0.24
   1  +6.0699927e+00  -5.9270232e+02  8.85e+01 1.61e+00 1.07e+02  2.0e-01  1.06
   2  +1.3208949e-01  -2.8384110e+02  6.92e+00 1.27e-01 8.37e+00  2.3e-02  1.06
   3  -5.9759550e+01  -1.2165790e+02  1.05e+00 1.92e-02 1.26e+00  3.6e-03  1.06
   4  -2.2714680e+02  -2.5992663e+02  5.09e-01 9.37e-03 6.16e-01  1.8e-03  1.06
   5  -3.9581036e+02  -4.0515085e+02  1.43e-01 2.62e-03 1.72e-01  5.1e-04  1.07
   6  -4.6216012e+02  -4.6258581e+02  6.31e-03 1.16e-04 7.62e-03  2.3e-05  1.07
   7

### Switching between linear solvers

In [8]:
# Solve the normal equations with CHOLMOD
tlp.params.KKTOptions = KKT.SolverOptions(
    KKT.CholmodSolver;
    normal_equations=true
);

In [9]:
Tulip.optimize!(tlp)


Problem info
  Name        : AFIRO
  Constraints : 27
  Variables   : 32
  Non-zeros   : 83

Reduced problem info
  Constraints : 25  (removed 2)
  Variables   : 32  (removed 0)
  Non-zeros   : 81  (removed 2)
Presolve time : 0.000s

Optimizer info
Linear solver options
  Arithmetic   : Float64
  Backend      : CHOLMOD - Cholesky
  System       : Normal equations

 Itn            PObj            DObj     PFeas    DFeas    GFeas       Mu  Time
   0  +8.5491918e+00  -6.2055378e+02  5.21e+02 9.00e+00 6.30e+02  1.0e+00  0.01
   1  +6.0699927e+00  -5.9270232e+02  8.85e+01 1.61e+00 1.07e+02  2.0e-01  0.71
   2  +1.3208949e-01  -2.8384110e+02  6.92e+00 1.27e-01 8.37e+00  2.3e-02  0.71
   3  -5.9759550e+01  -1.2165790e+02  1.05e+00 1.92e-02 1.26e+00  3.6e-03  0.72
   4  -2.2714680e+02  -2.5992663e+02  5.09e-01 9.37e-03 6.16e-01  1.8e-03  0.72
   5  -3.9581036e+02  -4.0515085e+02  1.43e-01 2.62e-03 1.72e-01  5.1e-04  0.72
   6  -4.6216012e+02  -4.6258581e+02  6.31e-03 1.16e-04 7.62e-03  2.3e-0

In [10]:
# Solve the augmented system with LDLFactorizations
tlp.params.KKTOptions = KKT.SolverOptions(
    KKT.LDLFact_SymQuasDef
);

In [11]:
Tulip.optimize!(tlp)


Problem info
  Name        : AFIRO
  Constraints : 27
  Variables   : 32
  Non-zeros   : 83

Reduced problem info
  Constraints : 25  (removed 2)
  Variables   : 32  (removed 0)
  Non-zeros   : 81  (removed 2)
Presolve time : 0.000s

Optimizer info
Linear solver options
  Arithmetic   : Float64
  Backend      : LDLFactorizations.jl
  System       : Augmented system

 Itn            PObj            DObj     PFeas    DFeas    GFeas       Mu  Time
   0  +8.5491918e+00  -6.2055378e+02  5.21e+02 9.00e+00 6.30e+02  1.0e+00  0.01
   1  +6.0699927e+00  -5.9270232e+02  8.85e+01 1.61e+00 1.07e+02  2.0e-01  0.69
   2  +1.3208949e-01  -2.8384110e+02  6.92e+00 1.27e-01 8.37e+00  2.3e-02  0.70
   3  -5.9759550e+01  -1.2165790e+02  1.05e+00 1.92e-02 1.26e+00  3.6e-03  0.70
   4  -2.2714680e+02  -2.5992663e+02  5.09e-01 9.37e-03 6.16e-01  1.8e-03  0.70
   5  -3.9581036e+02  -4.0515085e+02  1.43e-01 2.62e-03 1.72e-01  5.1e-04  0.70
   6  -4.6216012e+02  -4.6258581e+02  6.31e-03 1.16e-04 7.62e-03  2.3e

In [12]:
# Solve the normal equations with a Krylov method (PR #56)
tlp.params.KKTOptions = KKT.SolverOptions(
    KKT.KrylovPDSolver;
    method=Krylov.minres  # Use the MINRES method
);

In [13]:
Tulip.optimize!(tlp)


Problem info
  Name        : AFIRO
  Constraints : 27
  Variables   : 32
  Non-zeros   : 83

Reduced problem info
  Constraints : 25  (removed 2)
  Variables   : 32  (removed 0)
  Non-zeros   : 81  (removed 2)
Presolve time : 0.000s

Optimizer info
Linear solver options
  Arithmetic   : Float64
  Backend      : Krylov (minres)
  System       : Normal equations

 Itn            PObj            DObj     PFeas    DFeas    GFeas       Mu  Time
   0  +8.5491918e+00  -6.2055378e+02  5.21e+02 9.00e+00 6.30e+02  1.0e+00  0.04
   1  +6.0708409e+00  -5.9269998e+02  8.85e+01 1.61e+00 1.07e+02  2.0e-01  0.98
   2  +1.3154772e-01  -2.8388880e+02  6.93e+00 1.27e-01 8.37e+00  2.3e-02  0.99
   3  -5.9358155e+01  -1.2176637e+02  1.05e+00 1.94e-02 1.27e+00  3.6e-03  0.99
   4  -2.1826956e+02  -2.5269774e+02  5.39e-01 9.92e-03 6.52e-01  1.9e-03  0.99
   5  -4.0076754e+02  -4.1236735e+02  1.91e-01 3.51e-03 2.30e-01  7.0e-04  0.99
   6  -4.4361690e+02  -4.4723522e+02  5.54e-02 1.02e-03 6.70e-02  2.0e-04  

### Using custom routines

The [UnitBlockAngular](https://github.com/mtanneau/UnitBlockAngular.jl) package defines
* Custom data structure `UnitBlockAngularMatrix <: AbstractMatrix`
* Custom linear solver `UnitBlockAngularFactor <: AbstractKKTSolver`
* Specialized matrix-vector and factorization routines

In [14]:
tlp = Tulip.Model{Float64}()  # Double-precision
Tulip.load_problem!(tlp, "dat/DER_24_1024_43.mps")
tlp.params.OutputLevel = 1
tlp.params.Presolve = 0;

In [15]:
# Solve with default linear algebra
Tulip.optimize!(tlp)


Problem info
  Name        : 
  Constraints : 1048
  Variables   : 6493
  Non-zeros   : 146136

Optimizer info
Linear solver options
  Arithmetic   : Float64
  Backend      : CHOLMOD
  System       : Augmented system

 Itn            PObj            DObj     PFeas    DFeas    GFeas       Mu  Time
   0  +5.8378084e+04  -1.4745600e+05  4.29e+04 9.99e+02 2.06e+05  1.0e+00  0.00
   1  +1.9103365e+05  -1.3692291e+04  3.50e+03 8.17e+01 1.68e+04  8.4e-02  0.02
   2  +4.8472124e+04  -1.3235046e+05  1.52e+02 3.55e+00 7.30e+02  3.7e-03  0.03
   3  +3.2569271e+04  -1.1690203e+05  1.10e+02 2.57e+00 5.28e+02  2.7e-03  0.05
   4  +1.5178802e+04  -6.9405091e+04  7.98e+01 1.86e+00 3.83e+02  2.0e-03  0.07
   5  +7.0793000e+03  -1.7684416e+04  4.27e+01 9.96e-01 2.05e+02  1.1e-03  0.08
   6  +2.0952835e+03  -2.5591281e+03  1.03e+01 2.40e-01 4.94e+01  2.6e-04  0.09
   7  +1.5346392e+03  +1.2757655e+03  6.34e-01 1.48e-02 3.04e+00  1.6e-05  0.10
   8  +1.4992564e+03  +1.4197355e+03  1.95e-01 4.55e-03 9.37e

In [16]:
using UnitBlockAngular

In [17]:
# Store A as a unit block-angular matrix with 24 linking constraints,
# 72 linking variables, 6421 columns and 1024 blocks
tlp.params.MatrixOptions = Tulip.TLA.MatrixOptions(
    UnitBlockAngularMatrix,
    m0=24, n0=72, n=6421 , R=1024
)

# Select custom linear solver
tlp.params.KKTOptions = Tulip.KKT.SolverOptions(UnitBlockAngularFactor);

In [18]:
tlp.params.Presolve = 0
Tulip.optimize!(tlp)


Problem info
  Name        : 
  Constraints : 1048
  Variables   : 6493
  Non-zeros   : 146136

Optimizer info
Linear solver options
  Arithmetic   : Float64
  Backend      : UnitBlockAngular
  System       : Normal equations

 Itn            PObj            DObj     PFeas    DFeas    GFeas       Mu  Time
   0  +5.8378084e+04  -1.4745600e+05  4.29e+04 9.99e+02 2.06e+05  1.0e+00  0.28
   1  +1.9103365e+05  -1.3692291e+04  3.50e+03 8.17e+01 1.68e+04  8.4e-02  1.41
   2  +4.8472124e+04  -1.3235046e+05  1.52e+02 3.55e+00 7.30e+02  3.7e-03  1.42
   3  +3.2569271e+04  -1.1690203e+05  1.10e+02 2.57e+00 5.28e+02  2.7e-03  1.42
   4  +1.5178802e+04  -6.9405091e+04  7.98e+01 1.86e+00 3.83e+02  2.0e-03  1.42
   5  +7.0793000e+03  -1.7684416e+04  4.27e+01 9.96e-01 2.05e+02  1.1e-03  1.43
   6  +2.0952835e+03  -2.5591281e+03  1.03e+01 2.40e-01 4.94e+01  2.6e-04  1.43
   7  +1.5346392e+03  +1.2757655e+03  6.34e-01 1.48e-02 3.04e+00  1.6e-05  1.44
   8  +1.4992564e+03  +1.4197355e+03  1.95e-01 4.55e

Default linear algebra
```
  Backend      : CHOLMOD
  System       : Augmented system
  
 Itn            PObj            DObj     PFeas    DFeas    GFeas       Mu  Time
...
  33  +1.4743650e+03  +1.4743650e+03  2.00e-10 4.38e-11 9.60e-10  5.1e-15  0.54
Solver exited with status Trm_Optimal
```

Specialized linear algebra
```
  Backend      : UnitBlockAngular
  System       : Normal equations
  
 Itn            PObj            DObj     PFeas    DFeas    GFeas       Mu  Time
...
  33  +1.4743650e+03  +1.4743650e+03  2.01e-10 4.38e-11 9.60e-10  5.1e-15  0.14
Solver exited with status Trm_Optimal
```

265 lines of code --> up to 10x speedup on structured problems

# Implementation

| module | #loc |
|:--|---:|
| Core | 594
| Presolve | 1080
| KKT | 609
| IPM | 748
| Interfaces | 1011
| **Total** | **4240**

$\rightarrow$ Specialized linear algebra is only ~5% of total code 😱

## Model representation

3 representations of the same LP.

* Original (user-facing) LP

\begin{align}
    \min_{x} \ \ \ 
    & c^{T}x + c_{0}\\
    s.t. \ \ \ 
    & l_{r} \leq Ax \leq u_{r}\\
    & l_{x} \leq x \leq u_{x}
\end{align}

* Presolved problem: same as above, but some rows/columns removed

* Standard form LP (internal)

\begin{align}
    \min_{\tilde{x}} \ \ \ 
    & \tilde{c}^{T} \tilde{x} + \tilde{c}_{0}\\
    s.t. \ \ \ 
    & \tilde{A} \tilde{x} = \tilde{b}\\
    & 0 \leq \tilde{x} \leq \tilde{u}
\end{align}

Internal workflow is

1. User inputs the model and calls `optimize!`
2. Pre-solve and extract reduced problem
3. Convert reduced problem to standard form
4. Instantiate matrix
5. Instantiate KKT solver
6. Optimize
7. Post-solve

## Matrix instantiation

The (standard form) matrix $A$ is instantiated by calling
```julia
A = construct_matrix(Ta, m, n, aI, aJ, aV; kwargs...)
```

This method needs to be extended for custom matrix types `Ta`.

Example for `SparseMatrixCSC`:
```julia
construct_matrix(
    ::Type{SparseMatrixCSC}, m::Int, n::Int,
    aI::Vector{Int}, aJ::Vector{Int}, aV::Vector{Tv}
) where{Tv<:Real} = sparse(aI, aJ, aV, m, n)
```

In [19]:
m, n = 2, 4
aI = [1, 2, 2]
aJ = [1, 2, 3]
aV = [1.1, 2.2, 2.3]

A = construct_matrix(SparseMatrixCSC, m, n, aI, aJ, aV)

2×4 SparseMatrixCSC{Float64,Int64} with 3 stored entries:
  [1, 1]  =  1.1
  [2, 2]  =  2.2
  [2, 3]  =  2.3

In [20]:
Matrix(A)

2×4 Array{Float64,2}:
 1.1  0.0  0.0  0.0
 0.0  2.2  2.3  0.0

## Linear systems


Each IPM iteration comprises
* One factorization (expensive)
* A few triangular solves
* Several matrix-vector products
* Many vector operations

$\Longrightarrow$ Persistent data structures to re-use storage and factorization + dispatch

### Current implementation

Root type `AbstractKKTSolver` and 3 methods
* `KKT.setup` (once per `optimize!`): instantiate the KKT solver
* `KKT.update!(kkt, θ⁻¹, rp, rd)` (once per iteration): update data and factorization
* `KKT.solve!(Δx, Δy, kkt, ξp, ξd)` (multiple times per iteration): solve the augmented system

KKT solvers are chosen via the `KKTOptions` parameter
```julia
tlp.params.KKTOptions = KKT.SolverOptions(
    KKT.KrylovPDSolver;
    method=Krylov.minres
)
```

### `KKT.setup`


Called after presolve and matrix instantiation as

```julia
KKT.setup(params.KKTOptions.Ts, A; params.KKTOptions.options...)
```

If no specialized dispatch, defaults to
```
setup(Ts, args...; kwargs...) = Ts(args...; kwargs...)
```
 $\rightarrow$ `KKTOptions` is basically a factory
 
 Custom KKT solvers must implement
```julia
MyKKTSolver(A; kwargs...)
```

In [21]:
?KKT.setup

```
setup(T::Type{<:AbstractKKTSolver}, args...; kwargs...)
```

Instantiate a KKT solver object.


### `KKT.update!`

Called once per IPM iteration (unless numerical issues) as
```julia
KKT.update!(kkt, θ⁻¹, regP, regD)
```

After calling `update!`, `kkt` can be used to solve the augmented system

$$
    \begin{bmatrix}
        -(\Theta^{-1} + R_{p}) & A^{T}\\
        A & R_{d}
    \end{bmatrix}
    \begin{bmatrix}
        \Delta x\\
        \Delta y
    \end{bmatrix}
    =
    \begin{bmatrix}
        \xi_{p}\\
        \xi_{d}
    \end{bmatrix}
$$

Primal-dual regularizations are handled by the IPM algorithm.

Direct methods: update the factorization

Iterative methods: update the preconditioner

In [22]:
?KKT.update!

```
update!(kkt, θinv, regP, regD)
```

Update internal data and factorization/pre-conditioner.

After this call, `kkt` can be used to solve the augmented system

```
    [-(Θ⁻¹ + Rp)   Aᵀ] [dx] = [ξd]
    [   A          Rd] [dy]   [ξp]
```

for given right-hand sides `ξd` and `ξp`.

# Arguments

  * `kkt::AbstractKKTSolver{Tv}`: the KKT solver object
  * `θinv::AbstractVector{Tv}`: $θ⁻¹$
  * `regP::AbstractVector{Tv}`: primal regularizations
  * `regD::AbstractVector{Tv}`: dual regularizations

---

```
update!(kkt::Dense_SymPosDef{<:Real}, θ, regP, regD)
```

Compute normal equations system matrix and update Cholesky factorization.

Uses Julia's generic linear algebra.

---

```
update!(kkt::Dense_SymPosDef{<:BlasReal}, θ, regP, regD)
```

Compute normal equations system matrix and update Cholesky factorization.

Uses BLAS and LAPACK routines.

---

```
update!(kkt, θ, regP, regD)
```

Update LDLᵀ factorization of the augmented system.

Update diagonal scaling $\theta$, primal-dual regularizations, and re-compute     the factorization. Throws a `PosDefException` if matrix is not quasi-definite.

---

```
update!(kkt, θ, regP, regD)
```

Compute normal equation system matrix, and update the factorization.

---

```
update!(kkt, θ, regP, regD)
```

Update LDLᵀ factorization of the augmented system.

Update diagonal scaling $\theta$, primal-dual regularizations, and re-compute     the factorization. Throws a `PosDefException` if matrix is not quasi-definite.

---

```
update!(kkt::KrylovPDSolver, θ, regP, regD)
```

Update diagonal scaling $θ$ and primal-dual regularizations.


### `KKT.solve!`

Called multiple times per iteration.
Solves the augmented system.

# Some lessons learned

User-facing LP

\begin{align}
    \min_{x} \ \ \ 
    & c^{T}x + c_{0}\\
    s.t. \ \ \ 
    & l_{r} \leq Ax \leq u_{r}\\
    & l_{x} \leq x \leq u_{x}
\end{align}

Very similar in spirit to [MatrixOptInterface.jl](https://github.com/jump-dev/MatrixOptInterface.jl)

For fast build/modify & presolve, we need:
* Add/remove rows/columns/coefficients
* Access to individual coefficients (more rare)
* Ensure row/column index consistency
* Fast row-based _and_ column-based iteration over $A$

The initial version stored coefficients in a `Dict`...

which is very slow and actually not that convenient.

## Current implementation

Similar in principle to [SoPlex](https://soplex.zib.de/), we store two copies of $A$ in sync:
* Column-based: `acols::Vector{Col}`
* Row-based: `arows::Vector{Row}`

Redundant but simplifies the access.

Each row/column is a (sorted) sparse vector to speed up modifications
```julia
mutable struct RowOrCol{T}
    nzind::Vector{Int}
    nzval::Vector{T}
end
```

Row/column deletion is still slow because all the matrix has to shift.

## Future versions (?)

Improvement opportunities
1. Batch modifications (currently defaults to individual calls)
2. Recently discovered [DynamicSparseArrays.jl](https://github.com/atoptima/DynamicSparseArrays.jl)
3. Also enable customization of these data structures, for, e.g.:
  * Distributed models
  * Operator-based modeling
  * ...

⚠️ Modifying this part will interfere with presolve. ⚠️

# Future directions

## (convex) Quadratic programming

Quadratic objective, linear constraints

\begin{align}
    (QP) \ \ \ \min_{x} \ \ \ 
    & \frac{1}{2} x^{T}Qx + c^{T}x\\
    & A x = b\\
    & x \geq 0
\end{align}

Same algorithm, but augmented systems are of the form

$$
    \begin{bmatrix}
        -(Q + \Theta^{-1}) & A^{T}\\
        A & 0
    \end{bmatrix}
    \begin{bmatrix}
        \Delta x\\
        \Delta y
    \end{bmatrix}
    =
    \begin{bmatrix}
        b\\
        c
    \end{bmatrix}
$$

## Iterative linear solvers 🚧

WIP
* Initial infrastructure using [Krylov.jl](https://github.com/JuliaSmoothOptimizers/Krylov.jl): cf [#56](https://github.com/ds4dm/Tulip.jl/pull/56)
* Changes to the IPM needed for speed & numerics

## Specialized solvers

* Block-angular problems: large-scale stochastic LPs
* Integration with [StructJuMP.jl](https://github.com/StructJuMP/StructJuMP.jl) & [BlockDecomposition.jl](https://github.com/atoptima/BlockDecomposition.jl)

## GPU support

Most likely as part of a specialized linear solver.